In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [3]:
import gc
import cv2
from PIL import Image
import matplotlib.pyplot as plt

###########################################################################

from tqdm import tqdm

#########################################################################

import lightgbm  as lgb
import catboost as cbt
import xgboost as xgb
import sklearn.ensemble as ensem
import sklearn.linear_model as lm
import sklearn.svm as svm 
import sklearn.neighbors as neibs
import sklearn.naive_bayes  as nb
import sklearn.discriminant_analysis as danl 

##########################################################

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

##########################################################

import sklearn.decomposition as decomp
import sklearn.manifold as mnfld


2025-08-18 20:52:48.926735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755550368.936700       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755550368.939752       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 20:52:48.956448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.11/dist-packages/keras/api/preprocessing/image/__init__.py)

In [4]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
imgsize = 14
imgsize2 = 196
images = []
paths = train.id_code
for path in paths :
    img = cv2.imread(f'../input/train_images/{path}.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(imgsize,imgsize),interpolation=cv2.INTER_CUBIC)
    img = (img.reshape((imgsize2,))/255).tolist()
    images.append(img)


In [5]:
trainxdf =  pd.DataFrame(images)


In [6]:
#kernel : "linear" | "poly" | "rbf" | "sigmoid"
#kernel  | "cosine" | "precomputed"
kpcatrain = decomp.KernelPCA(n_components=8,
                            kernel="sigmoid",
                            gamma=None, 
                            degree=4, 
                            coef0=1,
                            kernel_params=None,
                            alpha=1.0, 
                            fit_inverse_transform=False, 
                            eigen_solver='auto',
                            tol=0,
                            max_iter=None,
                            remove_zero_eig=False,
                            random_state=0
                          )

colskpca = ["kpca1","kpca2","kpca3","kpca4","kpca5",
           "kpca6","kpca7","kpca8"]
kpcatraindf =  pd.DataFrame(kpcatrain.fit_transform(trainxdf),
                            columns=colskpca)

trainxdf =  pd.concat([trainxdf,kpcatraindf], axis=1)


NameError: name 'decomp' is not defined

In [7]:
rf = ensem.RandomForestClassifier(n_estimators=2000,
                                  max_depth=200,
                                  random_state=100)
rf.fit(trainxdf,train.diagnosis)


RandomForestClassifier(max_depth=200, n_estimators=2000, random_state=100)

In [8]:
imgsize = 14
imgsize2 = 196
imagest = []
paths = test.id_code
for path in paths :
    img = cv2.imread(f'../input/test_images/{path}.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(imgsize,imgsize),interpolation=cv2.INTER_CUBIC)
    img = (img.reshape((imgsize2,))/255).tolist()
    imagest.append(img)


In [9]:
testxdf =  pd.DataFrame(imagest)


In [10]:
testxdf.head()


,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,0.003922,0.000000,0.000000,0.007843,0.337255,0.454902,0.423529,0.447059,0.439216,0.537255,...,0.349020,0.360784,0.352941,0.368627,0.345098,0.392157,0.003922,0.000000,0.000000,0.003922
1,0.043137,0.054902,0.215686,0.215686,0.223529,0.215686,0.227451,0.247059,0.219608,0.231373,...,0.439216,0.470588,0.447059,0.384314,0.380392,0.388235,0.294118,0.321569,0.058824,0.047059
2,0.007843,0.007843,0.007843,0.352941,0.356863,0.368627,0.360784,0.368627,0.380392,0.384314,...,0.305882,0.286275,0.301961,0.290196,0.321569,0.313725,0.321569,0.352941,0.007843,0.007843
3,0.007843,0.007843,0.007843,0.419608,0.337255,0.329412,0.329412,0.274510,0.325490,0.329412,...,0.321569,0.325490,0.341176,0.333333,0.286275,0.301961,0.317647,0.007843,0.003922,0.003922
4,0.000000,0.000000,0.000000,0.227451,0.266667,0.290196,0.290196,0.286275,0.309804,0.341176,...,0.125490,0.156863,0.172549,0.164706,0.141176,0.109804,0.090196,0.000000,0.000000,0.000000


In [11]:
########################################################################

kpcatest = decomp.KernelPCA(n_components=8,
                            kernel="sigmoid",
                            gamma=None, 
                            degree=4, 
                            coef0=1,
                            kernel_params=None,
                            alpha=0.1, 
                            fit_inverse_transform=False, 
                            eigen_solver='auto',
                            tol=0,
                            max_iter=None,
                            remove_zero_eig=False,
                            random_state=0
                          )

colskpca = ["kpca1","kpca2","kpca3","kpca4","kpca5",
           "kpca6","kpca7","kpca8"]
kpcatestdf =  pd.DataFrame(kpcatest.fit_transform(testxdf),
                           columns=colskpca)
testxdf =  pd.concat([testxdf,kpcatestdf], axis=1)


NameError: name 'decomp' is not defined

In [12]:
predVal = rf.predict(testxdf)


In [13]:
test['diagnosis'] = predVal
test.to_csv("submission.csv",index=False, header=True)
